# Analyze Ad Images with Cortex AI

## Overview

This template shows you how to use Snowflake Cortex AI functions to work with unstructured image data. [cite: 32] You'll learn how to store images directly in Snowflake and use familiar SQL to classify them and generate new ad copy in just a few minutes. [cite: 5, 11, 12]

In [ ]:
/* setup role */
USE ROLE accountadmin;
CREATE ROLE IF NOT EXISTS ai_sql_demo_role;
GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE ai_sql_demo_role WITH GRANT OPTION;
GRANT CREATE DATABASE ON ACCOUNT TO ROLE ai_sql_demo_role WITH GRANT OPTION;
GRANT ROLE ai_sql_demo_role TO ROLE sysadmin;

/* setup database and schemas */
USE ROLE ai_sql_demo_role;
CREATE WAREHOUSE IF NOT EXISTS ai_sql_demo_wh WITH warehouse_size = SMALL;
USE WAREHOUSE ai_sql_demo_wh; 
CREATE DATABASE IF NOT EXISTS ai_sql_demo_db;
CREATE SCHEMA IF NOT EXISTS ai_sql_demo_db.admin;
CREATE STAGE IF NOT EXISTS ai_sql_demo_db.admin.ai_sql_stage DIRECTORY=(ENABLE=true) ENCRYPTION=(TYPE='SNOWFLAKE_SSE');
CREATE SCHEMA IF NOT EXISTS ai_sql_demo_db.impressions;

/* create table where impressions data will be added */
CREATE OR REPLACE TABLE ai_sql_demo_db.impressions.synthetic_impressions (
    Impression_ID NUMBER,
    Customer_ID STRING,
    Event_Timestamp TIMESTAMP_NTZ,
    Image_File_Name STRING,
    Image_Type STRING, 
    Was_Clicked BOOLEAN,
    IP_Address STRING,
    Latitude FLOAT,
    Longitude FLOAT,
    City STRING,
    State STRING
);

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras

## Step 2: Load images into a table

Now, let's load our sample ad images into a table. The `TO_FILE` function lets you work with unstructured data, like images, by creating a `FILE` data type. This query finds all PNG files in a public stage and loads their metadata into a table for analysis. 

In [ ]:
/* create a table where the images will be added */
CREATE OR REPLACE TABLE CORTEX_AI_IMAGE_DEMO.ADS.IMAGES AS
SELECT
    TO_FILE(FILE_URL) AS IMG_FILE,
    RELATIVE_PATH,
    FILE_URL
FROM
    DIRECTORY (@cortex_ai_image_demo.ads.image_stage)
WHERE
    RELATIVE_PATH LIKE '%.png'
    AND RELATIVE_PATH NOT LIKE '%instructions%';

Check the table & file datatype

In [ ]:
SELECT
    img_file,
    file_url,
    last_modified
FROM ai_sql_demo_db.impressions.images;

In [ ]:
DESCRIBE TABLE ai_sql_demo_db.impressions.images;

## Step 3: Classify ads with AI_CLASSIFY

Let's use `AI_CLASSIFY` to automatically categorize our ads. Provide the function with the image file and a list of categories, and it will return the best fit. This is great for sorting and organizing your unstructured image data.

SELECT
    IMG_FILE,
    AI_CLASSIFY(PROMPT('Classify the type of ad in this image {0}', IMG_FILE), ['product shot', 'lifestyle', 'promo offer']) :labels[0] ::TEXT AS CLASSIFICATION
FROM CORTEX_AI_IMAGE_DEMO.ADS.IMAGES;

## Step 4: Generate new taglines with AI_COMPLETE

Next, run this query to have `AI_COMPLETE` create a fresh advertising tagline for each image. By passing the image file to the `PROMPT` function, you can ask the model to perform tasks related to the image content.

In [ ]:
SELECT
    IMG_FILE,
    AI_COMPLETE('pixtral-large', PROMPT('Extract the text from this image and create an advertising tagline. Just return the tagline itself: {0}', IMG_FILE)) AS TAGLINE,
    FILE_URL,
    LAST_MODIFIED
FROM CORTEX_AI_IMAGE_DEMO.ADS.IMAGES;

## Step 5: Filter images with AI_FILTER

Finally on image, let's use `AI_FILTER` to find specific images. Past data suggests that ads with coffee makers perform well. This query uses `AI_FILTER` to check if an image contains a coffee maker, making it easy to select the right ads for your campaign.

In [ ]:
SELECT 
    IMG_FILE, 
    RELATIVE_PATH, 
    FILE_URL
FROM CORTEX_AI_IMAGE_DEMO.ADS.IMAGES
WHERE AI_FILTER(PROMPT('Does this image {0} have an espresso machine or a coffee pot?', IMG_FILE));

## Step 6: Aggregate Text Reviews

Finally, let's use `AI_AGG` functions to get insights from all customer reviews on those ads. We are interested what are the overall sentiment & top themes from those reviews.

In [ ]:
SELECT * FROM AI_SQL_DEMO_DB.IMPRESSIONS.IMPRESSIONS_WITH_REVIEWS;

In [ ]:
SELECT
    image_type,

    AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'score the sentiment of the feedback on a scale of 1 to 5 where 1 is very negative and 5 is very positive. Just write 1, 2, 3, 4, or 5. Return the average value only') as sentiment,

    AI_AGG('Image Type: '|| image_type || '\n\nFeedback: ' || impression_review, 'Analyze the feedback from these reviews and summarize the feedback in one sentence') as summary 
        
FROM AI_SQL_DEMO_DB.IMPRESSIONS.IMPRESSIONS_WITH_REVIEWS
GROUP BY image_type;

## Key Takeaways

* **Analyze Images with SQL**: You can use the `FILE` data type to store images and other unstructured data directly in Snowflake.
* **Use Cortex AI Functions**: Functions like `AI_CLASSIFY`, `AI_COMPLETE`, and `AI_FILTER` allow you to classify, generate content for, and filter images using simple SQL commands.
* **Simplify Your Workflow**: Analyzing image content can be done in a single, executable notebook without needing complex external tools or setup.

## Additional Resources

* [Documentation: Cortex AI SQL Functions](https://docs.snowflake.com/)
